<a href="https://colab.research.google.com/github/vikishan13/Imdb-Movie-reviews-Sentiment-Analysis/blob/main/Imdb_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
import tensorflow_datasets as tfds

In [ ]:
imdb, info = tfds.load("imdb_reviews", with_info = True, as_supervised=True)

In [ ]:
train_data, test_data = imdb['train'], imdb['test']

In [ ]:
import numpy as np

In [ ]:
xtrain = []
ytrain = []
xtest = []
ytest = []
for x, y in train_data:
  xtrain.append(x.numpy().decode('utf-8')) # Extracting text from tensorflow datatype
  ytrain.append(y.numpy())

for x,y in test_data:
  xtest.append(x.numpy().decode('utf-8'))
  ytest.append(y.numpy())

In [ ]:
xtrain[0]

"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."

In [ ]:
ytrain[:10]

[0, 0, 0, 1, 1, 1, 0, 0, 0, 0]

In [ ]:
# The values needs to be in numpy array before passing into Neural Networks
ytrain = np.array(ytrain)
ytest = np.array(ytest)

In [ ]:
ytrain[:10]

array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])

In [ ]:
# defining Hyperparameters for the model
vocab_size = 10000
embedding_dim = 16
max_length = 120

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = "<OOV>")

In [ ]:
tokenizer.fit_on_texts(xtrain)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(xtrain)
padded = pad_sequences(sequences, maxlen = max_length, truncating = 'post')

In [ ]:
test_sequences = tokenizer.texts_to_sequences(xtest)
test_padded = pad_sequences(test_sequences, maxlen = max_length, truncating = 'post')

In [ ]:
# building the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(padded, ytrain, epochs = 10, validation_data=(test_padded, ytest))
# Reminder: slight overfitting is noticed in the model, can use Early Stopping and Patience to control overfitting / Hyperparameter tunings need to be done

Epoch 1/10
782/782 [==============================] - 7s 7ms/step - loss: 0.5025 - accuracy: 0.7334 - val_loss: 0.3870 - val_accuracy: 0.8238
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2486 - accuracy: 0.9040 - val_loss: 0.3998 - val_accuracy: 0.8254
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.1107 - accuracy: 0.9695 - val_loss: 0.4922 - val_accuracy: 0.8124
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0312 - accuracy: 0.9957 - val_loss: 0.5899 - val_accuracy: 0.8105
Epoch 5/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0090 - accuracy: 0.9992 - val_loss: 0.6630 - val_accuracy: 0.8113
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0037 - accuracy: 0.9996 - val_loss: 0.7362 - val_accuracy: 0.8069
Epoch 7/10
782/782 [==============================] - 6s 8ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.7927 - val_accuracy: 0.8083
Epoch 

In [ ]:
text = ['i watched this movie which was absolutely superb and lovely to watch and i loved the movie very much']
text_sequences = tokenizer.texts_to_sequences(text)
pred = pad_sequences(text_sequences, maxlen = max_length, truncating = 'post')

In [ ]:
value = model.predict(pred)
print(value)
print("positive review") if value > 0.5 else print("negative negative")

1/1 [==============================] - 0s 97ms/step
[[0.9958978]]
positive review


In [ ]:
text = ['i watched this movie which was terrific and waste of money to watch and i hated this movie and those directors were taken this movie just for timesake']
text_sequences = tokenizer.texts_to_sequences(text)
pred = pad_sequences(text_sequences, maxlen = max_length, truncating = 'post')

In [ ]:
value = model.predict(pred)
print(value)
print("positive review") if value > 0.5 else print("negative review")

1/1 [==============================] - 0s 20ms/step
[[0.00279332]]
negative review
